# Introduction 

...

Before you get started, run the code cell below to set everything up.

In [ ]:
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon

import folium
from folium import Choropleth
from folium.plugins import HeatMap, MarkerCluster

You'll use the `embed_map()` function to visualize your maps.

In [ ]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# Exercises

### 1) Create a buffer.

62 hospitals

In [ ]:
hospitals = gpd.read_file("../input/geospatial-course-data/nyu_2451_34494/nyu_2451_34494.shp")
hospitals['latitude'] = hospitals.geometry.to_crs(epsg=4326).y
hospitals['longitude'] = hospitals.geometry.to_crs(epsg=4326).x
hospitals.head()

In [ ]:
m = folium.Map(location=[40.7, -74], zoom_start=11) 

mc = MarkerCluster()

for idx, row in hospitals.iterrows():
    if not math.isnan(row['longitude']) and not math.isnan(row['latitude']):
        mc.add_child(folium.Marker([row['latitude'], row['longitude']]))
        folium.Circle([row['latitude'], row['longitude']],
                    radius=1000,
                    color='blue',
                    fill=True
                    ).add_to(m)
        
m.add_child(mc)
m 

### 2)

notes on [buffer](https://shapely.readthedocs.io/en/latest/manual.html#object.buffer)

In [ ]:
hosp_buffer = gpd.GeoDataFrame(hospitals, geometry=hospitals.geometry.buffer(1000))
hosp_buffer.head()

In [ ]:
m = folium.Map(location=[40.7, -74], zoom_start = 11) 

HeatMap(data=hospitals[['latitude', 'longitude']], radius=20, max_zoom=13).add_to(m)

m 

In [ ]:
#[MultiPolygon(list(buffer_gdf.geometry.values)).buffer(0).contains(i) for i in hospitals_gdf.geometry]

In [ ]:
collisions_df = pd.read_csv("../input/geospatial-course-data/NYPD_Motor_Vehicle_Collisions.csv")
collisions_gdf = gpd.GeoDataFrame(collisions_df, geometry=gpd.points_from_xy(collisions_df["LONGITUDE"], collisions_df["LATITUDE"]))
collisions_gdf.head()

note to self. not every collision needs hospitalization

In [ ]:
danger_gdf = collisions_gdf[collisions_gdf[['NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED']].sum(axis=1)>0]

In [ ]:
from time import time

m = folium.Map(location=[40.7, -74], zoom_start=11) 

mc = MarkerCluster()

for idx, row in hospitals.iterrows():
    if not math.isnan(row['longitude']) and not math.isnan(row['latitude']):
        folium.Circle([row['latitude'], row['longitude']],
                    radius=1000,
                    color='blue',
                    fill=True
                    ).add_to(m)

start = time()

for idx, row in danger_gdf.iterrows():
    if not math.isnan(row['LONGITUDE']) and not math.isnan(row['LATITUDE']):
        mc.add_child(folium.Marker([row['LATITUDE'], row['LONGITUDE']]))
m.add_child(mc)

end = time()

print(end-start)

In [ ]:
embed_map(m, "m_1.html")

In [ ]:
from tqdm import tqdm
mp = MultiPolygon(list(hosp_buffer.geometry.values)).buffer(0)
inside_range = [mp.contains(i) for i in tqdm(danger_gdf.geometry.values)]

In [ ]:
outside_danger_gdf = danger_gdf[[not i for i in inside_range]]
len(outside_danger_gdf)

In [ ]:
from time import time

m = folium.Map(location=[40.7, -74], zoom_start=11) 

mc = MarkerCluster()

for idx, row in hospitals.iterrows():
    if not math.isnan(row['longitude']) and not math.isnan(row['latitude']):
        folium.Circle([row['latitude'], row['longitude']],
                    radius=1000,
                    color='blue',
                    fill=True
                    ).add_to(m)

start = time()

for idx, row in outside_danger_gdf.iterrows():
    if not math.isnan(row['LONGITUDE']) and not math.isnan(row['LATITUDE']):
        mc.add_child(folium.Marker([row['LATITUDE'], row['LONGITUDE']]))
m.add_child(mc)

end = time()

print(end-start)

In [ ]:
embed_map(m, "m2.html")

initially made heatmap with count column all ones. bad idea again because all accidents treated equally, whereas we know some are more dangerous than others

In [ ]:
m = folium.Map(location=[40.7, -74], zoom_start = 11) 

danger_gdf_copy = danger_gdf.copy()
danger_gdf_copy['count'] = danger_gdf_copy[[
    #'NUMBER OF PERSONS INJURED', 
    'NUMBER OF PERSONS KILLED',
    #'NUMBER OF PEDESTRIANS INJURED', 
    'NUMBER OF PEDESTRIANS KILLED',
    #'NUMBER OF CYCLIST INJURED', 
    'NUMBER OF CYCLIST KILLED',
    #'NUMBER OF MOTORIST INJURED', 
    'NUMBER OF MOTORIST KILLED'
    ]].sum(axis=1)

HeatMap(data=danger_gdf_copy[['LATITUDE', 'LONGITUDE', 'count']].groupby(['LATITUDE', 'LONGITUDE']).sum().reset_index().values.tolist(), radius=20, max_zoom=1).add_to(m)

for idx, row in hospitals.iterrows():
    if not math.isnan(row['longitude']) and not math.isnan(row['latitude']):
        folium.Circle([row['latitude'], row['longitude']],
                    radius=1000,
                    color='blue'
                    ).add_to(m)

embed_map(m) 

what percentage of car accidents w/ at least one injury or fatality were within 1 km of a hospital?
you can add x new hospitals.  how to increase this number to greater than some percentage?

[closest point](https://gis.stackexchange.com/questions/222315/geopandas-find-nearest-point-in-other-dataframe)